# Numerical Integration

## Quadrature

**Quadrature** is the process of determining the area under a curve by approximating it with geometric shapes. The term originates from the idea of finding a square with an equivalent area to a given region.

There are many numerical techniques to perform quadrature, but we will focus on four fundamental methods:

* **Trapezoid Rule** – Approximates the area using trapezoids.  
* **Simpson’s Rule** – Uses parabolic segments for improved accuracy.  
* **Romberg Integration** – Refines the Trapezoidal Rule using Richardson extrapolation.  
* **Gaussian Quadrature** – Achieves high accuracy by strategically choosing evaluation points and weights.

Each of these methods is based on the fundamental principle of approximating an integral as a weighted sum of function values:

$$
\int_a^b f(x) dx \approx \sum_{k=1}^N f(x_k) \omega_k
$$

where:

* $N = \frac{b - a}{h}$ is the number of integration steps,  
* $h = x_{k+1} - x_k$ is the step size between successive points,  
* $\omega_k$ represents weighting factors that depend on the chosen quadrature method.

The choice of method and the number of steps $N$ directly impact the accuracy of numerical integration. Simpler methods like the Trapezoid Rule work well for smooth functions but may require many steps for precision. More advanced techniques, such as Gaussian Quadrature, can achieve high accuracy with fewer evaluations by carefully selecting points $x_k$ and weights $\omega_k$.

In the following sections, we will explore each method, discussing their derivations, ad
vantages, and applications.
## Trapezoid Rule

One of the simplest numerical integration methods is the **Trapezoid Rule**, which approximates the area under a curve by dividing it into a series of trapezoids. Instead of approximating the function with rectangles (as in Riemann sums), we use trapezoids to achieve a more accurate estimate of the integral.

The integral we seek to approximate is:

$$I(a, b) = \int_a^b f(x) dx$$

The area of a single trapezoid, with base width $h$ and heights given by function values at two consecutive points, is:

$$A_k = \frac{1}{2} h [ f(a + (k - 1)h) + f(a + kh) ]$$

Summing over all $N$ trapezoids from $x = a$ to $x = b$, we obtain the Trapezoid Rule approximation:

$$I(a, b) \approx \frac{1}{2} h \sum_{k=1}^N \left[ f(a + (k - 1)h) + f(a + kh) \right]$$

Rearranging the summation, we can rewrite this as:

$$I(a, b) \approx h \left[ \frac{1}{2} f(a) + \frac{1}{2} f(b) + \sum_{k=1}^{N-1} f(a + kh) \right]$$

This formulation highlights an important computational advantage:  
- The function values at the interior points are summed once.  
- The function values at the endpoints ($a$ and $b$) are weighted by $\frac{1}{2}$ to avoid double-counting.

The Trapezoid Rule provides a straightforward yet effective method for numerical integration, especially when dealing with smooth functions. However, for highly curved functions, higher-order methods like **Simpson’s Rule** may offer improved accuracy with fewer steps.


In [ ]:
import integrate as intg
import numpy as np
import matplotlib.pyplot as plt

# Define a test function
def f(x):
    return np.sin(x)
# Integration limits
a, b = 0, np.pi
# Number of steps
N = 8
# Compute integrals
trap_result = intg.trapezoidrule(f, a, b, N)
# Exact integral value for comparison
exact_value = 2
# Print results
print(f"Trapezoidal Rule: {trap_result} (Error: {abs(trap_result - exact_value)})")
fx = np.linspace(a,b,100)
x = np.linspace(a, b, N + 1)

plt.plot(fx,f(fx),'k',label = 'sin(x)')
plt.plot(x, f(x), 'r', label='Points')
plt.fill_between(x, f(x), alpha=0.3, label="Trapezoidal Area")
plt.scatter(x, f(x), color='blue')
#plt.gca().set_aspect('equal')
plt.legend(loc='upper left')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Trapezoidal Rule Approximation')
plt.grid()
plt.show()

## Simpson’s Rule

While the Trapezoid Rule approximates the function with linear segments, **Simpson’s Rule** takes it a step further by fitting the function with a quadratic interpolation. This provides a more accurate estimate, especially for functions that exhibit curvature.

### Quadratic Interpolation

To derive Simpson’s Rule, we approximate the function using a quadratic polynomial:

$f(x) = Ax^2 + Bx + C$

Given function values at three points $x = -h, 0, h$, we express them in terms of the unknown coefficients:

$f(-h) = Ah^2 - Bh + C$,  
$f(0) = C$,  
$f(h) = Ah^2 + Bh + C$

Solving for the coefficients:

$A = \frac{1}{h^2} \left[ \frac{1}{2} f(-h) - f(0) + \frac{1}{2} f(h) \right]$,  
$B = \frac{1}{2h} [f(h) - f(-h)]$,  
$C = f(0)$.

With this quadratic form, we can now integrate the function over the interval.

### Derivation of Simpson’s Rule

The integral of the quadratic approximation over a symmetric interval $[-h, h]$ is:

$$\int_{-h}^{h} (Ax^2 + Bx + C) dx = \frac{2}{3} Ah^3 + 2Ch.$$

Substituting the coefficients and simplifying, we obtain:

$$\int_{-h}^{h} f(x) dx \approx \frac{1}{3} h \left[ f(-h) + 4f(0) + f(h) \right].$$

### Generalizing Simpson’s Rule

For a general integral $I(a, b) = \int_a^b f(x) dx$, where the interval $[a, b]$ is divided into an **even** number of subintervals, we apply the above formula at successive points:

$$I(a, b) \approx \frac{1}{3} h \left[ f(a) + 4f(a + h) + 2f(a + 2h) + 4f(a + 3h) + 2f(a + 4h) + \dots + f(b) \right].$$

Observing the pattern, we can express this as:

$$I(a, b) \approx \frac{1}{3} h \left[ f(a) + f(b) + 4 \sum_{k \text{ odd}}^{N-1} f(a + kh) + 2 \sum_{k \text{ even}}^{N-2} f(a + kh) \right],$$

or equivalently,

$$I(a, b) \approx \frac{1}{3} h \left[ f(a) + f(b) + 4 \sum_{k=1}^{N/2} f(a + (2k - 1)h) + 2 \sum_{k=1}^{N/2 - 1} f(a + 2kh) \right].$$

### Key Takeaways

- **Simpson’s Rule is a weighted sum** of function values at different points, alternating between coefficients of **4** and **2**.
- **It requires an even number of subintervals** ($N$ must be even).
- **It provides a higher accuracy** than the Trapezoid Rule for the same step size, as it incorporates curvature information.

By using quadratic interpolation, Simpson’s Rule significantly improves integration accuracy, making it especially useful for smooth functions with non-linear behavior.


In [ ]:
import integrate as intg
import numpy as np
import matplotlib.pyplot as plt

# Define the function
def f(x):
    return np.sin(x)**2

# Integration limits
a, b = 0, np.pi
N = 10  # Number of intervals (should be even for Simpson’s rule)
exact_value = np.pi/2
simp_result = intg.simpsonrule(f,a,b,N)
print(f"Simpson Rule: {simp_result} (Error: {abs(simp_result - exact_value)})")
# Generate x values
x = np.linspace(a, b, N + 1)
fx = np.linspace(a, b, 1000)  # High-resolution x values

plt.plot(fx,f(fx), 'r', label="f(x) = sin(x)")

# Simpson’s Rule Approximation with Parabolic Interpolation
for i in range(0, N, 2):
    x_sample = x[i:i+3]  # Take 3 points at a time (Simpson’s rule works in pairs)
    y_sample = f(x_sample)

    # Fit a quadratic polynomial (degree 2)
    coeffs = np.polyfit(x_sample, y_sample, 2)
    
    # Generate x values for smooth parabolic curves
    x_curve = np.linspace(x_sample[0], x_sample[-1], 50)
    y_curve = np.polyval(coeffs, x_curve)

    # Plot the parabolic segment
    plt.plot(x_curve, y_curve, 'b--')
    plt.fill_between(x_curve, y_curve, alpha=0.3,color='blue')
# Scatter plot interpolation points
plt.scatter(x, f(x), color='black', label="Interpolation Points")

# Labels and title
plt.xlabel("x")
plt.ylabel("f(x)")
plt.legend(loc = 'upper left')
plt.title("Simpson's Rule Approximation with Parabolic Interpolation")
plt.grid()
plt.show()


## How Accurate Are These Methods?

When using numerical integration, two main sources of error affect accuracy:

- **Approximation Error**: The inherent error from approximating an integral numerically.
- **Rounding Error**: The error due to finite precision in computer arithmetic.

For most numerical integrations, the total error can be estimated as:

$$\epsilon \approx C I(a, b), \quad \text{where } C \approx 10^{-16}$$

### Errors in the Trapezoid Rule

It can be shown (see pages 149-151 of *Computational Physics*) that the approximation error for the Trapezoid Rule follows the **Euler-Maclaurin formula**:

$$\epsilon \approx h^2 \frac{1}{12} [f'(a) - f'(b)]$$

Since the Trapezoid Rule integrates linear functions exactly and has an error term proportional to $h^2$, it is classified as a **first-order method** (accurate to $h$). 

To minimize the combined effects of approximation and rounding errors, an optimal step size $h$ and number of steps $N$ must be chosen:

$$h^2 \frac{1}{12} [f'(a) - f'(b)] \sim C I(a, b)$$

which leads to an estimate for the required number of steps:

$$N \approx (b - a) \sqrt{\frac{[f'(a) - f'(b)]}{12 I(a, b) C}}$$

### Errors in Simpson’s Rule

A more detailed derivation shows that the approximation error for **Simpson’s Rule** is:

$$\epsilon \approx h^4 \frac{1}{90} [f'''(a) - f'''(b)]$$

This means Simpson’s Rule is a **third-order integration method** (accurate to $h^3$), offering significantly improved precision over the Trapezoid Rule.

Again, balancing approximation and rounding errors gives an estimate for the required number of steps:

$$N \approx (b - a) \sqrt{\frac{[f'''(a) - f'''(b)]}{90 I(a, b) C}}$$

### That’s All Great—IF We Knew the Function!

A valid concern! In practice, we rarely compute derivatives to estimate integration errors. Instead, a much simpler approach is used:

1. **Compute the integral with $N_1$ steps** (step size $h_1$), so that the true value of the integral $I$ is:

   $I_1 + \epsilon = I_1 + ch_1^2 = I$

2. **Compute the integral with twice the number of steps** ($N_2 = 2N_1$), reducing the step size to $h_2 = \frac{1}{2} h_1$:

   $I_2 + \epsilon = I_2 + ch_2^2 = I$

3. **Use these two results to estimate the error**:

   - **For the Trapezoid Rule**: $\epsilon_2 = \frac{1}{3} (I_2 - I_1)$
   - **For Simpson’s Rule**: $\epsilon_2 = \frac{1}{15} (I_2 - I_1)$

This technique allows for adaptive refinement of integration accuracy **without needing explicit derivatives of the function**.


# Romberg Integration and the Finer Points

## From Above

Using the Trapezoid or Simpson’s method, we can estimate the error in an integral by comparing calculations with different step sizes. If we compute the integral using $N_1$ steps (step size $h_1$) and then with $N_2 = 2N_1$ steps (step size $h_2 = \frac{1}{2} h_1$), the errors are:

$$
\epsilon_2 = \frac{1}{3} (I_2 - I_1) \quad \text{(Trapezoid Rule)}
$$

$$
\epsilon_2 = \frac{1}{15} (I_2 - I_1) \quad \text{(Simpson’s Rule)}
$$

By iterating this process, we can refine the calculation until the estimated error $\epsilon_i$ reaches a desired tolerance, where:

$$
\epsilon_i = \frac{1}{3} (I_i - I_{i-1})
$$

However, note that half the terms in $I_i$ are already present in $I_{i-1}$, meaning we can optimize our computations.

## Adaptive Integration Techniques

To improve efficiency, we express $I_i$ in terms of $I_{i-1}$ using the Trapezoid Rule:

$$
I_i = h_i \left[ \frac{1}{2} f(a) + \frac{1}{2} f(b) + \sum_{k=1}^{N_i - 1} f(a + kh_i) \right]
$$

Rewriting the sum:

$$
I_i = h_i \left[ \frac{1}{2} f(a) + \frac{1}{2} f(b) + \sum_{k \text{ even}}^{N_i - 2} f(a + kh_i) + \sum_{k \text{ odd}}^{N_i - 1} f(a + kh_i) \right]
$$

Since the even-indexed terms in $I_i$ correspond to the terms in $I_{i-1}$, we simplify:

$$
\sum_{k \text{ even}}^{N_i - 2} f(a + kh_i) = \sum_{k=1}^{N_i / 2 - 1} f(a + 2kh_i) = \sum_{k=1}^{N_{i-1} - 1} f(a + kh_{i-1})
$$

Thus, the integral simplifies to:

$$
I_i = \frac{1}{2} I_{i-1} + h_i \sum_{k \text{ odd}}^{N_i - 1} f(a + kh_i)
$$

### Steps for Adaptive Integration:

1. Choose an initial number of steps and compute $I_1$.
2. Double the number of steps and calculate $I_2$ and the error $\epsilon_2$.
3. If the error is within the specified tolerance, stop; otherwise, repeat step 2.

## Using Simpson’s Rule

A similar optimization applies to Simpson’s Rule. Define:

$$
S_i = \frac{1}{3} [f(a) + f(b) + 2 \sum_{k \text{ even}}^{N_i - 2} f(a + kh_i)]
$$

$$
T_i = \sum_{k \text{ odd}}^{N_i - 1} f(a + kh_i)
$$

Since $S_i = S_{i-1} + T_{i-1}$, the integral at iteration $i$ is:

$$
I_i = h_i (S_i + 2T_i)
$$

## Romberg Integration – We Can Do Even Better!

The Trapezoid Rule error is $\epsilon_i = ch_i^2 = \frac{1}{3} (I_i - I_{i-1})$, so the true integral is:

$$
I = I_i + \frac{1}{3} [I_i - I_{i-1}] + O(h_i^4)
$$

This expression improves accuracy to third order, with a fourth-order error!

To refine our notation, let $R_{i,1} = I_i$, then:

$$
R_{i,2} = I_i + \frac{1}{3} [I_i - I_{i-1}] = R_{i,1} + \frac{1}{3} (R_{i,1} - R_{i-1,1})
$$

More generally:

$$
R_{i,m+1} = R_{i,m} + \frac{1}{4^m - 1} (R_{i,m} - R_{i-1,m})
$$

with the error:

$$
\epsilon_m = \frac{1}{4^m - 1} (R_{i,m} - R_{i-1,m})
$$

## Romberg Integration Algorithm

1. Compute the first two estimates using the Trapezoid Rule: $I_1 = R_{1,1}$ and $I_2 = R_{2,1}$.
2. Compute the more accurate estimate $R_{2,2} = R_{2,1} + \frac{1}{3} (R_{2,1} - R_{1,1})$.
3. Compute the next trapezoidal estimate $I_3 = R_{3,1}$ and use it to compute $R_{3,2}$ and $R_{3,3}$.
4. Continue for $I_i = R_{i,1}$, refining to $R_{i,m}$ until the error is within tolerance.

A typical refinement sequence:

- $I_1 \equiv R_{1,1}$
- $I_2 \equiv R_{2,1} \rightarrow R_{2,2}$
- $I_3 \equiv R_{3,1} \rightarrow R_{3,2} \rightarrow R_{3,3}$
- $I_4 \equiv R_{4,1} \rightarrow R_{4,2} \rightarrow R_{4,3} \rightarrow R_{4,4}$

las**



In [ ]:
import integrate as intg
import numpy as np
import matplotlib.pyplot as plt

# Define the function to integrate
def f(x):
    return np.sin(x)

# Integration limits
a, b = 0, np.pi
N = 5  # Number of iterations (controls accuracy)

# Compute Romberg integration
romberg_result = intg.rombergrule(f, a, b, N)

# Exact integral for comparison
exact_value = 2

# Create a sequence of increasing iterations
N_values = np.arange(2, 2 * N, 2)
romberg_estimates = [intg.rombergrule(f, a, b, n) for n in N_values]

# Plot error convergence
errors = np.abs(np.array(romberg_estimates) - exact_value)

plt.plot(N_values, errors, 'bo-', label="Romberg Error")
plt.yscale("log")  # Log scale to visualize error reduction
plt.xlabel("Iterations (N)")
plt.ylabel("Absolute Error")
plt.title("Convergence of Romberg Integration")
plt.grid()
plt.legend()
plt.show()

# Print results
print(f"Romberg's Rule Estimate: {romberg_result}")
print(f"Exact Value: {exact_value}")
print(f"Final Error: {abs(romberg_result - exact_value)}")


## Higher-Order Integration Techniques

**Lagrange Interpolating Polynomial:** Given $N$ points, we can fit a polynomial of degree $N-1$. This leads to **Newton-Cotes formulas**, which generalize Simpson’s Rule to higher-order polynomials (see *Computational Physics*, p. 164).

Another approach is **Gaussian Quadrature**, which optimally chooses weights and evaluation points to integrate a $(2N-1)$th-degree polynomial with just $N$ function evaluations.

## Newton-Cotes Quadrature

The **Newton-Cotes formulas** provide a family of numerical integration methods that approximate an integral using polynomials fitted through equally spaced points. These include:

- **Trapezoid Rule** (degree 1 polynomial)
- **Simpson’s Rule** (degree 2 polynomial)
- **Boole’s Rule** (degree 4 polynomial)
- **Higher-order Newton-Cotes formulas**

# Gaussian Quadrature

## Motivation: Why Gaussian Quadrature?

In previous methods like the **Trapezoid Rule** and **Simpson’s Rule**, we evaluated the function at evenly spaced points and computed the integral as a weighted sum of these values. These methods work well but are not always the most **efficient**.

Gaussian Quadrature improves upon this by:
- Choosing **both the evaluation points and the weights optimally** rather than using evenly spaced intervals.
- Providing **higher accuracy with fewer function evaluations** compared to equally spaced quadrature methods.
- Allowing integration of higher-order polynomials exactly with fewer points.

Instead of using fixed intervals, Gaussian Quadrature finds the **best possible** nodes $x_i$ and corresponding weights $\omega_i$ so that the integral:

$$
I = \int_a^b f(x) dx
$$

is approximated as:

$$
I \approx \sum_{i=1}^n \omega_i f(x_i)
$$

where:
- $x_i$ are the **quadrature nodes** (carefully chosen points in the integration range).
- $\omega_i$ are the **weights** assigned to each function evaluation.

## Gaussian Quadrature and Orthogonal Polynomials

The key idea behind Gaussian Quadrature is that **polynomial functions of degree $2n-1$ or lower can be integrated exactly using only $n$ points**. This is accomplished by choosing $x_i$ as the **roots of a special orthogonal polynomial**.

For example:
- **Legendre polynomials** for standard **Gauss-Legendre Quadrature**.
- **Chebyshev polynomials** for **Gauss-Chebyshev Quadrature**.
- **Laguerre polynomials** for integrals over semi-infinite domains.
- **Hermite polynomials** for Gaussian-weighted integrals.

The integral is computed using the roots of these polynomials as evaluation points.

### Gauss-Legendre Quadrature

For standard integrals of the form:

$$
I = \int_{-1}^{1} f(x) dx
$$

the **Legendre polynomials** $P_n(x)$ determine the quadrature nodes $x_i$ as their roots. The weights $\omega_i$ are chosen such that any polynomial of degree up to $2n-1$ is integrated exactly.

For an arbitrary integral over $[a, b]$, we perform a **change of variables**:

$$
x = \frac{b-a}{2} t + \frac{b+a}{2}
$$

which transforms the integral into:

$$
I = \frac{b-a}{2} \int_{-1}^{1} f\left( \frac{b-a}{2} t + \frac{b+a}{2} \right) dt
$$

and then apply Gauss-Legendre Quadrature.

### Example: Two-Point Gauss-Legendre Quadrature

For **$n=2$**, the nodes and weights are:

- $x_1 = -\frac{1}{\sqrt{3}}, x_2 = \frac{1}{\sqrt{3}}$
- $\omega_1 = \omega_2 = 1$

Thus, the integral approximation is:

$$
I \approx f\left(-\frac{1}{\sqrt{3}}\right) + f\left(\frac{1}{\sqrt{3}}\right)
$$

which integrates any **polynomial up to degree 3 exactly**.

### Higher-Order Gauss-Legendre Quadrature

For **$n=3$**, the nodes and weights are:

- $x_1 = -\sqrt{\frac{3}{5}}, x_2 = 0, x_3 = \sqrt{\frac{3}{5}}$
- $\omega_1 = \omega_3 = \frac{5}{9}, \omega_2 = \frac{8}{9}$

Giving the approximation:

$$
I \approx \frac{5}{9} f\left(-\sqrt{\frac{3}{5}}\right) + \frac{8}{9} f(0) + \frac{5}{9} f\left(\sqrt{\frac{3}{5}}\right)
$$

which integrates any **polynomial up to degree 5 exactly**.

## Other Types of Gaussian Quadrature

Different weight functions result in different quadrature methods:

1. **Gauss-Chebyshev Quadrature**  
   - Uses **Chebyshev polynomials**.
   - Useful for integrals with weight functions of the form $(1-x^2)^{-1/2}$.

2. **Gauss-Laguerre Quadrature**  
   - Uses **Laguerre polynomials**.
   - Ideal for integrals of the form $\int_0^\infty e^{-x} f(x) dx$.

3. **Gauss-Hermite Quadrature**  
   - Uses **Hermite polynomials**.
   - Used for integrals involving the weight function $e^{-x^2}$, common in **probability and physics applications**.

## Why Use Gaussian Quadrature?

- **High accuracy with fewer evaluations**: It integrates polynomials up to degree $2n-1$ exactly.
- **Efficient for smooth functions**: When the function is well-behaved, Gaussian Quadrature outperforms methods like Simpson’s Rule.
- **Widely used in scientific computing**: It is a cornerstone in **finite element methods**, **quantum mechanics**, **signal processing**, and many other fields.

### When Not to Use Gaussian Quadrature?

- If the function has **singularities or discontinuities**, standard Gaussian Quadrature may not work well. Alternative methods like **adaptive quadrature** or **Gauss-Kronrod Quadrature** may be better suited.
- If a large number of points is needed, computing **high-degree orthogonal polynomials** and their roots can be computationally expensive.

---

## Summary

| Method                  | Nodes Chosen By | Integrates Exactly Up To | Use Case |
|-------------------------|----------------|--------------------------|----------|
| Gauss-Legendre         | Legendre roots | Degree $2n-1$ | General definite integrals |
| Gauss-Chebyshev        | Chebyshev roots | Degree $2n-1$ | Weight function $(1-x^2)^{-1/2}$ |
| Gauss-Laguerre         | Laguerre roots | Degree $2n-1$ | Semi-infinite domains, $e^{-x}$ weight |
| Gauss-Hermite          | Hermite roots | Degree $2n-1$ | Gaussian-weighted integrals |

Gaussian Quadrature provides an efficient and powerful method for numerical integration, significantly outperforming traditional rules when applied to smooth functions.

---


To understand a bit more about Gaussian Quadrature, let's take a look at the roots of these special polynomials.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp

# Function to plot orthogonal polynomials and their roots
def plot_polynomial_roots(poly_func, n, title, x_range=(-1, 1)):
    x = np.linspace(x_range[0], x_range[1], 400)
    poly = poly_func(n)  # Get the polynomial function
    y = poly(x)  # Evaluate polynomial at x points

    # Compute roots (quadrature points)
    roots = poly.roots

    # Plot polynomial
    plt.figure(figsize=(8, 5))
    plt.plot(x, y, label=f'{title} (Degree {n})', linewidth=2)
    plt.axhline(0, color='black', linewidth=1, linestyle='--')
    
    # Plot roots
    plt.scatter(roots, np.zeros_like(roots), color='red', zorder=3, label="Zeros (Quadrature Points)", s=80)

    # Formatting
    plt.title(f"{title} and Its Zeros")
    plt.xlabel("x")
    plt.ylabel("P(x)")
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot Legendre Polynomial and its roots (Gauss-Legendre)
plot_polynomial_roots(sp.legendre, n=5, title="Legendre Polynomial P5(x)")

# Plot Laguerre Polynomial and its roots (Gauss-Laguerre)
plot_polynomial_roots(sp.laguerre, n=5, title="Laguerre Polynomial L5(x)", x_range=(0, 13))

# Plot Hermite Polynomial and its roots (Gauss-Hermite)
plot_polynomial_roots(sp.hermite, n=5, title="Hermite Polynomial H5(x)", x_range=(-5, 5))

## Algorithm for Gaussian Quadrature
The code below is an attempt at illustrating what is happening in gaussian quadrature. While I did do some checking, I used ChatGPT refine my code and something is still amiss, can you find it?

In [ ]:
# Gaussian Quadrature Example
def gaussian_quadrature(f, n, quad_type="legendre", a=None, b=None):
    """
    Computes numerical integration using Gaussian quadrature.
    
    Parameters:
    - f: Function to integrate
    - n: Number of quadrature points
    - quad_type: Type of quadrature ('legendre', 'laguerre', 'hermite')
    - a, b: Limits of integration (only needed for 'legendre')
    
    Returns:
    - Approximate integral value
    """
    if quad_type == "legendre":
        if a is None or b is None:
            raise ValueError("Legendre quadrature requires limits a and b.")
        nodes, weights = np.polynomial.legendre.leggauss(n)
        x_transformed = 0.5 * (b - a) * nodes + 0.5 * (a + b)
        integral = 0.5 * (b - a) * np.sum(weights * f(x_transformed))
    elif quad_type == "laguerre":
        nodes, weights = sp.l_roots(n)
        integral = np.sum(weights * f(nodes))
    elif quad_type == "hermite":
        nodes, weights = sp.h_roots(n)
        integral = np.sum(weights * f(nodes))
    else:
        raise ValueError("Invalid quadrature type. Choose 'legendre', 'laguerre', or 'hermite'.")
    return integral

# Define function to integrate
f = lambda x: np.exp(x)  # Change function as needed

# Gauss-Legendre: Integral of e^x from 0 to 2
result_legendre = gaussian_quadrature(f, n=5, quad_type="legendre", a=0, b=2)
print(f"Legendre Approximation (0 to 2): {result_legendre}")

# Gauss-Laguerre: Integral of e^(-x) * e^x = 1 (Should return ~1)
result_laguerre = gaussian_quadrature(f, n=5, quad_type="laguerre")
print(f"Laguerre Approximation (0 to ∞): {result_laguerre}")

# Gauss-Hermite: Integral of e^(-x^2) * e^x
result_hermite = gaussian_quadrature(f, n=5, quad_type="hermite")
print(f"Hermite Approximation (-∞ to ∞): {result_hermite}")

# Integration in Python
Python has a entire library devoted to algorthims implementing the integration techniques that we talked about today. You just need to import *scipy.integrate*

In [1]:
import scipy.integrate as sci

The second line will list how the contents and summary of the functions within the module *scipy.integrate*. Also note that I have included my own integration algorithms (**user beware**) in myint.py on the *github* repository *CompProbSol*

The next thing to do is figure out how to use these functions. My functions in `myint` all accept *lambda* functions. The functions in `integrate` accept lambda functions or data. The built-in functions are also coded to take either arrays or *lambda* functions (depending on the function) and arrays of where the y values are evaluated or limits. Below are some examples using the trapezoid rule, Simpson's rule, and Romberg integration.

For these examples, we'll switch it up and evaluate an integral that has great importance in statistical mechanics,
$$ \int_a^b \frac{x^3}{e^{x}-1} dx $$

In [ ]:
import myint
import numpy as np
x = np.linspace(1,8,100)
ftrap = lambda x: x**3/(np.exp(x)-1)
farray = ftrap(x)
t =myint.trapezoidrule(ftrap,1,8,100)
p = sci.trapezoid(farray,x)
print(t)
print(p)

In [ ]:
t=myint.simpsonrule(ftrap,1,8,1000)
p = sci.simps(farray,x=x)
print(t)
print(p)

In [ ]:
t=myint.rombergrule(ftrap,1,8)
p = sci.romberg(ftrap,1,8)
print(t)
print(p)

Python also has builtin functions to deal with double and triple integrals. Below are examples of double and triple integrals using *dblquad* and *tplquad*

Many times in electrostatics and magnetostatics, we need to integrate using non constant limits of integration; use lambda functions to accomplish this in Python
`f2` is the result from the integral: $$\int_{0}^{3}\int_{0}^{1-2x} x^2 y$$ Note that the limits are inputted in the reverse order of what you may think.

In [ ]:
f2 = lambda x, y: x**2+y
sci.dblquad(f2,0,3, lambda x: 0, lambda x: 1-2*x)

The triple integral below is $$\int_{0}^{2}\int_{0}^{3-x}\int_{1}^{4-x-y}x^2y+3z$$

In [ ]:
f3 = lambda x,y,z: x**2*y+3*z
sci.tplquad(f3,0,2,lambda x: 0, lambda x: 3-x, lambda x,y:1,lambda x,y: 4-x-y)

## Problem 7: Anharmonic Oscillator (From *Computational Physics* Newman)
The simple harmonic oscillator crops up in many places. Its behaviors can be studied readily using analytic methods and it has the important property that its period of oscillation is constant, independent of its amplitude, making it useful, for instance, for keeping time in watches and clocks. 
Frequently in physics, however, we also come across anharmonic oscillators, whose period varies with amplitude and whose behavior cannot usually be calculated analytically. A general classical oscillator can be thought of as a particle in a concave potential well. When disturbed, the particle will rock back and forth in the well. The harmonic oscillator corresponds to a quadratic potential $V(x)\sim x^2$. Any other form gives an anharmonic oscillator.
One way to calculate the motion of an oscillator is to write down the equation for the conservation of energy in the system. If the particle has a mass *m* and position *x*, then the total energy is equal to the sum of the kinetic and potential energies thus: $$E = \frac{1}{2}m\left(\frac{dx}{dt}\right)^2 + V(x).$$ Since the energy must be constant over time, this equation is effectively a differential equation linking *x* and *t*. 
Let us assume that the potential $V(x)$ is symmetric about $x=0$ and let us set our anharmonic oscillator going with amplitude *a* and it swings back towards the origin. Then at $t=0$ we have $dx/dt=0$.

(a) Convince yourself that the period of this oscillator is $$T = \int_{0}^{a} \frac{dx}{\sqrt{V(a)-V(x)}}$$.

(b) Write a program in Python that takes the amplitude *a* and a lambda function for $V(x)$ as arguments and calculates the period of an harmonic/anharmonic oscillator with a mass $m=1$. Use a potential form $V(x) = x^2$, $V(x)=x^4$, and the Morse potential $V(r) = D_e(1-e^{-a(r-r_e)^2})$, where $D_e$ is the well depth and $a = \sqrt{k_e/2D_e}$, $k_e$ represents the force constant at the minimum of the well. Experiment with different integral functions.

(c) Use your function to make a graph of the period for amplitudes ranging from $a=0$ to $a=4$. Do the results match your physical intuition?

(d) If we extend consideration to 3 dimensions where $V(x)\rightarrow V(r)\sim r^2$ and only consider radial motion, we see that the results are the same as the 1 dimensional case. What happens when we consider angular momentum? Repeat (a)-(c) in the case of 3D motion with angular momentum about the equilibrium point of a spherical pendulum. 